# Pandas - Payments

In [84]:
import numpy as np
import pandas as pd

### 1.- Data Loading

Load the data into an analysis-ready format.
— Possibly convert the data to an intermediate data storage format (CSV, HDF5,
SQL, FITS, ENDF, ENSDF).
— Convert the data to an easy-access data structure (NumPy arrays, Pandas data
frames).

In [85]:
# Localización del fichero a cargar
file_path = '/home/alberto/Master/PythonDataScience/hvtasfp.csv'

In [86]:
# Nombre de las columnas (si el fichero no incluye cabecera)
column_names = ['Store', 'Date', 'Till', 'Ticket', 'Method', 'Hour', 'Type', 'Amount']

In [87]:
# Cargamos el fichero
%time df = pd.read_csv(file_path, header=None, names=column_names)

CPU times: user 22.6 s, sys: 7.12 s, total: 29.7 s
Wall time: 36.3 s


In [88]:
# Comprobamos el tamaño del df
df.shape

(63536741, 8)

### 2.- Data Cleaning and Preparation

In [89]:
# Echamos un vistazo a las primeras filas para ver que pinta
# tiene el dataset
df.head()

,Store,Date,Till,Ticket,Method,Hour,Type,Amount
0,851,20170301,1,24461,19,90721,1,33.32
1,851,20170301,1,24536,19,90721,5,33.32
2,851,20170301,1,25077,19,81206,1,8.99
3,851,20170301,1,25168,19,81206,5,8.99
4,851,20170301,1,25396,19,84418,1,22.39


### 2.1. Handling Missing Data

Gestionamos valores nulos que puedan existir en el dataset.
Contamos con los siguientes métodos para la limpieza del dataset

* dropna
* fillna
* isnull
* notnull

In [90]:
# Comprobamos si existen valores nulos en algunas de las columnas
df.isnull().any()

Store     False
Date      False
Till      False
Ticket    False
Method    False
Hour      False
Type      False
Amount    False
dtype: bool

### 2.2 Data Transformation

Realizamos operaciones de filtrado, limpieza y otras transformaciones sobre el dataset

#### 2.2.1 Removing Duplicates

Comprobamos si existen registros duplicados

In [91]:
# filtramos aquellos registros que se encuentran duplicados
# y obtenemos el número de ellos. Utilizamos df.duplicated()
# al cuál le podemos pasar el argumento keep para que nos devuelva
# la información completa o sólo el registro duplicado (primero o 
# último)
df.duplicated().any()

True

In [93]:
# echamos un vistazo a algunos de los registros duplicados
df[df.duplicated() == True].head()

,Store,Date,Till,Ticket,Method,Hour,Type,Amount
491851,7661,20170308,1,28356,1,165032,1,29.99
1543794,1860,20170322,2,1360,15,210145,1,19.99
2017396,7944,20170329,2,89,15,171136,1,7.00
2623324,796,20170406,3,52400,1,183058,1,17.50
2750262,7006,20170407,1,13026,1,181242,1,20.00


In [94]:
# contamos cuantos tenemos (utilizamos una columna sólo para que 
# no nos devuelva el total de todas las columnas repetidas)
df[df.duplicated() == True]['Store'].count() 

149

In [95]:
# determinamos si queremos o no conservar los duplicados, en nuestro
# caso, los vamos a eliminar del dataset
df.drop_duplicates(inplace=True)

In [96]:
# verificamos el tamaño del df (antes: 63536741, 8)
df.shape 

(63536592, 8)

#### 2.2.2. Transforming Data Using a Function or Mapping

In [97]:
# Determinamos los tipos de operación
df['Type'].unique()

array([1, 5])

In [98]:
# Para que sea más entendible, vamos a transformar los códigos en
# dos tipos de operación: 1.- Venta, 5.- Devolucion
# Creamos un diccionario con la correspondencia
type_to_string = {1:'Sale', 5:'Return'}

In [99]:
# Creamos una nueva columna a la que llamos operación y utilizamos
# la función map para pasarle a la columna type el diccionario con 
# la correspondencia y que nos devuelva esta en la columna 'Operation'
df['Operation'] = df['Type'].map(type_to_string) 

In [100]:
# Nos sobra ya la columna Type, por lo que podemos eliminarla del df
df.drop(columns='Type', inplace=True)

In [101]:
df.head()

,Store,Date,Till,Ticket,Method,Hour,Amount,Operation
0,851,20170301,1,24461,19,90721,33.32,Sale
1,851,20170301,1,24536,19,90721,33.32,Return
2,851,20170301,1,25077,19,81206,8.99,Sale
3,851,20170301,1,25168,19,81206,8.99,Return
4,851,20170301,1,25396,19,84418,22.39,Sale


In [102]:
# Revisamos el campo de Método de Pago
df['Method'].unique()

array([19,  6, 14,  8, 17,  1,  9, 15,  5, 10, 18,  7,  2, 20,  3])

In [103]:
# Para que sea más entendible, vamos a transformar los métodos de pago
# en texto. A diferencia  de con el tipo, aquí disponemos de una tabla
# la descripción de cada una de las formas de pago, así que vamos a 
# usarla para ello haciendo un join más adelante

In [104]:
# El campo Amount vamos a ponerlo en negativo cuando se trate
# de una operación de devolución
#  -  filtramos aquellas operaciones de devolución, devolviendo la 
#     columna 'Amount' : df.loc[df['Operation'] == 'Return', 'Amount']
#  -  modificamos el valor con "map" aplicando una función lambda para 
#     multiplicar * -1 : .map(lambda x: x * -1)
df.loc[df['Operation'] == 'Return', 'Amount'] = df.loc[df['Operation'] == 'Return', 'Amount'].map(lambda x: x * -1)

In [105]:
# Ya tenemos los importes de las devoluciones en negativo
df.head(10)

,Store,Date,Till,Ticket,Method,Hour,Amount,Operation
0,851,20170301,1,24461,19,90721,33.32,Sale
1,851,20170301,1,24536,19,90721,-33.32,Return
2,851,20170301,1,25077,19,81206,8.99,Sale
3,851,20170301,1,25168,19,81206,-8.99,Return
4,851,20170301,1,25396,19,84418,22.39,Sale
5,851,20170301,1,25401,19,84418,-22.39,Return
6,851,20170301,1,26045,19,80854,-20.99,Return
7,851,20170301,1,26138,19,80854,20.99,Sale
8,851,20170301,1,26540,19,72725,34.98,Sale
9,851,20170301,1,26591,19,70244,19.99,Sale


#### 2.2.3. Date columns

Vamos a transformar las columnas de fecha y hora, a Series de tipo TimeStamp para faciliar su manejo

In [ ]:
# Convertimos el campo Date en uno TimeStamp
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [77]:
df['Date'].describe()

count                63536592
unique                    526
top       2018-01-04 00:00:00
freq                   362146
first     2017-03-01 00:00:00
last      2018-08-08 00:00:00
Name: Date, dtype: object

In [79]:
# echamos un vistazo al df
df.head()

,Store,Date,Till,Ticket,Method,Hour,Amount,Operation
0,851,2017-03-01,1,24461,19,90721,33.32,Sale
1,851,2017-03-01,1,24536,19,90721,-33.32,Return
2,851,2017-03-01,1,25077,19,81206,8.99,Sale
3,851,2017-03-01,1,25168,19,81206,-8.99,Return
4,851,2017-03-01,1,25396,19,84418,22.39,Sale


In [81]:
df['Hour'].describe()

count    6.353659e+07
mean     1.607768e+05
std      3.445439e+04
min      0.000000e+00
25%      1.311530e+05
50%      1.638310e+05
75%      1.915380e+05
max      4.758250e+05
Name: Hour, dtype: float64

In [82]:
pd.to_datetime(df['Hour'], format='%H%M%S')

ValueError: time data 52 does not match format '%H%M%S' (match)

In [83]:
df.iloc[52]

Store                        851
Date         2017-03-01 00:00:00
Till                           1
Ticket                     31215
Method                         6
Hour                      102739
Amount                     49.98
Operation                   Sale
Name: 52, dtype: object